In [30]:
import os, subprocess
import xml.sax
import mwparserfromhell 
import json
import re
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import sys
import os
from multiprocessing import Pool 
import tqdm 
from itertools import chain
from functools import partial
import gc
import json

In [31]:
dumps = os.listdir('wiki_dump/')
os.stat('wiki_dump/' + dumps[14]).st_size

4096

In [32]:
input_titles = pd.read_csv('small_seed_data/math_articles_8687', sep='\t', encoding='utf-8')

In [33]:
input_titles = input_titles.iloc[:,0].tolist()

In [34]:
data_path = 'wiki_dump/' + dumps[0]

In [47]:
dumps[0]

'26_enwiki-20180720-pages-articles20.xml-p18754736p20254736.bz2'

In [48]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._articles = []
        self._total_articles = 0
        self._articles_count = 0

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ''.join(self._buffer)

        if name == 'page':
            self._total_articles += 1
            # Search through the page to see if the page is a target_article
            article = process_article(**self._values)
            # Append to the list of articles
            if article:
                self._articles_count += 1
                self._articles.append(article)

In [68]:
def process_article(title, text, timestamp):
    """Search wiki_dump for desired articles"""
    wiki = mwparserfromhell.parse(text)
    if title in input_titles:
        stripped_text = wiki.strip_code().strip()
        headings = wiki.filter_headings()
        wikilinks = []
        for link in wiki.filter_wikilinks():
            link = link.strip('[]')
            if 'File:' not in link and \
            'Category:' not in link and \
            'Wikipedia:' not in link and \
            'en:' not in link:
                wikilinks.append(link)
        return {'title': title,\
                'stripped_text': stripped_text,\
                'full_text': text,\
                'wikilinks': wikilinks}

In [69]:
start = timer()

handler = WikiXmlHandler()

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    if (i + 1) % 10000 == 0:
        sys.stdout.write('\r' +f'Processed {i + 1} lines so far.')
    try:
        parser.feed(line)
    except StopIteration:
        break
#     if handler._articles_count > 1:
#         break
    
end = timer()


Processed 23830000 lines so far.

In [70]:
print(f'\nSearched through {handler._total_articles} articles.')
timey = round((end - start) / 60)
print(f'\nFound {handler._articles_count} articles in {timey} minutes.')
print(f'The estimate of searching the entire dump is {round(55 * timey / 60, 2)} hours without paralellization')


Searched through 457859 articles.

Found 189 articles in 48 minutes.
The estimate of searching the entire dump is 44.0 hours without paralellization


In [71]:
articles = handler._articles

In [64]:
handler._articles_count

189

In [65]:
print(articles[1]['full_text'])

[[Image:Parallelogramoid.svg|thumb|right|Levi-Civita's parallelogramoid]]
In the [[mathematics|mathematical]] field of [[differential geometry]], the '''Levi-Civita parallelogramoid''' is a [[quadrilateral]] in a [[curved space]] whose construction generalizes that of a [[parallelogram]] in the [[Euclidean plane]].  It is named for its discoverer, [[Tullio Levi-Civita]]. Like a parallelogram, two opposite sides ''AA''&prime; and ''BB''&prime; of a parallelogramoid are parallel (via [[parallel transport]] {{Not a typo|along }} side ''AB'') and the same length as each other, but the fourth side ''A''&prime;''B''&prime; will not in general be parallel to or the same length as the side ''AB,'' although it will be straight (a [[geodesic]]).

==Construction==
A parallelogram in [[Euclidean geometry]] can be constructed as follows:
* Start with a straight line segment ''AB'' and another straight line segment ''AA''&prime;.
* Slide the segment ''AA''&prime; along ''AB'' to the endpoint ''B'', 

In [67]:
with open('data/temp_results/file1.json', 'w') as fout:
    for page in handler._articles:
        fout.write(json.dumps(page) + '\n')
    

TypeError: Object of type 'Heading' is not JSON serializable

In [ ]:
with open(out_dir, 'w') as fout:
            # Write as json
            for book in handler._books:
                fout.write(json.dumps(book) + '\n')

In [67]:
def cycle_articles(x):
    for i in x:
        yield(i)

In [68]:
article = cycle_articles(articles)

In [75]:
next(article)

('Regular chain',
 "In computer algebra, a regular chain is a particular kind of triangular set in a multivariate polynomial ring over a field. It enhances the notion of characteristic set. \n \n == Introduction == \n \n Given a linear system, one can convert it to a triangular system via Gaussian elimination. For the non-linear case, given a polynomial system F over a field, one can convert (decompose or triangularize) it to a finite set of triangular sets, in the sense that the algebraic variety V(F) is described by these triangular sets. \n \n A triangular set may merely describe the empty set. To fix this degenerated case, the notion of regular chain was introduced, independently by Kalkbrener (1993), Yang and Zhang (1994). Regular chains also appear in Chou and Gao (1992). Regular chains are special triangular sets which are used in different algorithms for computing unmixed-dimensional decompositions of algebraic varieties. Without using factorization, these decompositions have b

In [80]:
articles[7]

('List of mathematical series',
 'This list of mathematical series contains formulae for finite and infinite sums. It can be used in conjunction with other tools for evaluating sums. \n *Here,  < math > 0^0 < /math >  is taken to have the value  < math > 1 < /math > \n * < math > B_n(x) < /math >  is a Bernoulli polynomial. \n * < math > B_n < /math >  is a Bernoulli number, and here,  < math > B_1=-\\frac{1}{2}. < /math > \n * < math > E_n < /math >  is an Euler number. \n * < math > \\zeta(s) < /math >  is the Riemann zeta function. \n * < math > \\Gamma(z) < /math >  is the gamma function. \n * < math > \\psi_n(z) < /math >  is a polygamma function. \n * < math > \\operatorname{Li}_s(z) < /math >  is a polylogarithm. \n * < math >  n \\choose k  < /math >  is binomial coefficient \n \n ==Sums of powers== \n See Faulhaber\'s formula. \n * < math > \\sum_{k=0}^m k^{n-1}=\\frac{B_n(m+1)-B_n}{n} < /math > \n The first few values are: \n * < math > \\sum_{k=1}^m k=\\frac{m(m+1)}{2} < /ma

In [14]:
handler2 = WikiXmlHandler()

parser2 = xml.sax.make_parser()

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    parser2.feed(line)
    
    if len(handler2._articles) > 3:
        break

In [15]:
handler2._articles[0]

IndexError: list index out of range

In [27]:
lines = []

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)
    if i > 5e5:
        break
        

In [28]:
data_path

'wiki_dump/26_enwiki-20180720-pages-articles20.xml-p18754736p20254736.bz2'

In [29]:
from bs4 import BeautifulSoup as bs

lines = []
for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)

soup = bs(b'\n'.join(lines).decode('utf-8'), 'lxml')

import wiki_parse as w
lines = w.get_lines_bz2('wiki_dump/' + dumps[4])
next(lines)
dumps[4]
import glob
dumps = glob.glob('wiki_dump/*.bz2')
lines = w.get_lines_bz2(dumps[4])
next(lines)

KeyboardInterrupt: 

In [16]:
print('This list of mathematical series contains formulae for finite and infinite sums. It can be used in conjunction with other tools for evaluating sums. \n *Here,  < math > 0^0 < /math >  is taken to have the value  < math > 1 < /math > \n * < math > B_n(x) < /math >  is a Bernoulli polynomial. \n * < math > B_n < /math >  is a Bernoulli number, and here,  < math > B_1=-\\frac{1}{2}. < /math > \n * < math > E_n < /math >  is an Euler number. \n * < math > \\zeta(s) < /math >  is the Riemann zeta function. \n * < math > \\Gamma(z) < /math >  is the gamma function. \n * < math > \\psi_n(z) < /math >  is a polygamma function. \n * < math > \\operatorname{Li}_s(z) < /math >  is a polylogarithm. \n * < math >  n \\choose k  < /math >  is binomial coefficient \n \n ==Sums of powers== \n See Faulhaber\'s formula. \n * < math > \\sum_{k=0}^m k^{n-1}=\\frac{B_n(m+1)-B_n}{n} < /math > \n The first few values are: \n * < math > \\sum_{k=1}^m k=\\frac{m(m+1)}{2} < /math > \n * < math > \\sum_{k=1}^m k^2=\\frac{m(m+1)(2m+1)}{6}=\\frac{m^3}{3}+\\frac{m^2}{2}+\\frac{m}{6} < /math > \n * < math > \\sum_{k=1}^m k^3 \n \n =\\left[\\frac{m(m+1)}{2}\\right]^2=\\frac{m^4}{4}+\\frac{m^3}{2}+\\frac{m^2}{4} < /math > \n \n See zeta constants. \n * < math > \\zeta(2n)=\\sum^{\\infty}_{k=1} \\frac{1}{k^{2n}}=(-1)^{n+1} \\frac{B_{2n} (2\\pi)^{2n}}{2(2n)!}  < /math > \n The first few values are: \n * < math > \\zeta(2)=\\sum^{\\infty}_{k=1} \\frac{1}{k^2}=\\frac{\\pi^2}{6} < /math >  (the Basel problem) \n * < math > \\zeta(4)=\\sum^{\\infty}_{k=1} \\frac{1}{k^4}=\\frac{\\pi^4}{90} < /math > \n * < math > \\zeta(6)=\\sum^{\\infty}_{k=1} \\frac{1}{k^6}=\\frac{\\pi^6}{945} < /math > \n \n ==Power series== \n \n ===Low-order polylogarithms=== \n Finite sums: \n * < math > \\sum_{k=0}^{n} z^k = \\frac{1-z^{n+1}}{1-z} < /math > , (geometric series) \n * < math > \\sum_{k=1}^n k z^k = z\\frac{1-(n+1)z^n+nz^{n+1}}{(1-z)^2} < /math > \n * < math > \\sum_{k=1}^n k^2 z^k = z\\frac{1+z-(n+1)^2z^n+(2n^2+2n-1)z^{n+1}-n^2z^{n+2}}{(1-z)^3}  < /math > \n * < math > \\sum_{k=1}^n k^m z^k = \\left(z \\frac{d}{dz}\\right)^m \\frac{1-z^{n+1}}{1-z} < /math > \n \n Infinite sums, valid for  < math > |z| < 1 < /math >  (see polylogarithm): \n * < math > \\operatorname{Li}_n(z)=\\sum_{k=1}^{\\infty} \\frac{z^k}{k^n} < /math > \n The following is a useful property to calculate low-integer-order polylogarithms recursively in closed form: \n * < math > \\frac{\\mathrm{d}}{\\mathrm{d}z}\\operatorname{Li}_n(z)=\\frac{\\operatorname{Li}_{n-1}(z)}{z} < /math > \n * < math > \\operatorname{Li}_{1}(z)=\\sum_{k=1}^\\infty \\frac{z^k}{k}=-\\ln(1-z) < /math > \n * < math > \\operatorname{Li}_{0}(z)=\\sum_{k=1}^\\infty z^k=\\frac{z}{1-z} < /math > \n * < math > \\operatorname{Li}_{-1}(z)=\\sum_{k=1}^\\infty k z^k=\\frac{z}{(1-z)^2} < /math > \n * < math > \\operatorname{Li}_{-2}(z)=\\sum_{k=1}^\\infty k^2 z^k=\\frac{z(1+z)}{(1-z)^3} < /math > \n * < math > \\operatorname{Li}_{-3}(z)=\\sum_{k=1}^\\infty k^3 z^k =\\frac{z(1+4z+z^2)}{(1-z)^4} < /math > \n * < math > \\operatorname{Li}_{-4}(z)=\\sum_{k=1}^\\infty k^4 z^k =\\frac{z(1+z)(1+10z+z^2)}{(1-z)^5} < /math > \n \n ===Exponential function=== \n * < math > \\sum_{k=0}^\\infty \\frac{z^k}{k!} = e^z < /math > \n * < math > \\sum_{k=0}^\\infty k\\frac{z^k}{k!} = z e^z < /math >  (cf. mean of Poisson distribution) \n * < math > \\sum_{k=0}^\\infty k^2 \\frac{z^k}{k!} = (z + z^2) e^z < /math >  (cf. second moment of Poisson distribution) \n * < math > \\sum_{k=0}^\\infty k^3 \\frac{z^k}{k!} = (z + 3z^2 + z^3) e^z < /math > \n * < math > \\sum_{k=0}^\\infty k^4 \\frac{z^k}{k!} = (z + 7z^2 + 6z^3 + z^4) e^z < /math > \n * < math > \\sum_{k=0}^\\infty k^n \\frac{z^k}{k!} = z \\frac{d}{dz} \\sum_{k=0}^\\infty k^{n-1} \\frac{z^k}{k!}\\,\\! = e^z T_{n}(z)  < /math > \n \n where  < math > T_{n}(z) < /math >  is the Touchard polynomials. \n \n ===Trigonometric, inverse trigonometric, hyperbolic, and inverse hyperbolic functions=== \n \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^k z^{2k+1}}{(2k+1)!}=\\sin z < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{z^{2k+1}}{(2k+1)!}=\\sinh z < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^k z^{2k}}{(2k)!}=\\cos z < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{z^{2k}}{(2k)!}=\\cosh z < /math > \n * < math > \\sum_{k=1}^\\infty \\frac{(-1)^{k-1}(2^{2k}-1)2^{2k}B_{2k}z^{2k-1}}{(2k)!}=\\tan z, |z| < \\frac{\\pi}{2} < /math > \n * < math > \\sum_{k=1}^\\infty \\frac{(2^{2k}-1)2^{2k}B_{2k}z^{2k-1}}{(2k)!}=\\tanh z, |z| < \\frac{\\pi}{2} < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^k2^{2k}B_{2k}z^{2k-1}}{(2k)!}=\\cot z, |z| < \\pi < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{2^{2k}B_{2k}z^{2k-1}}{(2k)!}=\\coth z, |z| < \\pi < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^{k-1}(2^{2k}-2)B_{2k}z^{2k-1}}{(2k)!}=\\csc z, |z| < \\pi < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{-(2^{2k}-2)B_{2k}z^{2k-1}}{(2k)!}=\\operatorname{csch} z, |z| < \\pi < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^kE_{2k}z^{2k}}{(2k)!}=\\operatorname{sech} z, |z| < \\frac{\\pi}{2} < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{E_{2k}z^{2k}}{(2k)!}=\\sec z, |z| < \\frac{\\pi}{2} < /math > \n * < math > \\sum_{k=1}^\\infty \\frac{(-1)^{k-1} z^{2k}}{(2k)!}=\\operatorname{ver}z < /math >  (versine) \n * < math > \\sum_{k=1}^\\infty \\frac{(-1)^{k-1} z^{2k}}{2(2k)!}=\\operatorname{hav}z < /math > < ref name= " Weisstein_hav " >  < /ref >  (haversine) \n * < math > \\sum_{k=0}^\\infty \\frac{(2k)!z^{2k+1}}{2^{2k}(k!)^2(2k+1)}=\\arcsin z, |z|\\le1 < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^k(2k)!z^{2k+1}}{2^{2k}(k!)^2(2k+1)}=\\operatorname{arcsinh} {z}, |z|\\le1 < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{(-1)^kz^{2k+1}}{2k+1}=\\arctan z, |z| < 1 < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{z^{2k+1}}{2k+1}=\\operatorname{arctanh} z, |z| < 1 < /math > \n * < math > \\ln2+\\sum_{k=1}^\\infty \\frac{(-1)^{k-1}(2k)!z^{2k}}{2^{2k+1}k(k!)^2}=\\ln\\left(1+\\sqrt{1+z^2}\\right), |z|\\le1 < /math > \n \n ===Modified-factorial denominators=== \n * < math > \\sum^{\\infty}_{k=0} \\frac{(4k)!}{2^{4k} \\sqrt{2} (2k)! (2k+1)!} z^k = \\sqrt{\\frac{1-\\sqrt{1-z}}{z}}, |z| < 1 < /math > < ref name= " gfo " >  < /ref > \n * < math > \\sum^{\\infty}_{k=0} \\frac{2^{2k} (k!)^2}{(k+1) (2k+1)!} z^{2k+2} = \\left(\\arcsin{z}\\right)^2, |z|\\le1  < /math > < ref name= " gfo " / > \n * < math > \\sum^{\\infty}_{n=0} \\frac{\\prod_{k=0}^{n-1}(4k^2+\\alpha^2)}{(2n)!} z^{2n} + \\sum^{\\infty}_{n=0} \\frac{\\alpha \\prod_{k=0}^{n-1}[(2k+1)^2+\\alpha^2]}{(2n+1)!}  z^{2n+1} = e^{\\alpha \\arcsin{z}}, |z|\\le1  < /math > \n \n === Binomial coefficients === \n * < math > (1+z)^\\alpha = \\sum_{k=0}^\\infty {\\alpha \\choose k} z^k, |z| < 1 < /math >  (see Binomial theorem) \n * < ref name= " ctcs " >  < /ref >   < math > \\sum_{k=0}^\\infty {{\\alpha+k-1} \\choose k} z^k = \\frac{1}{(1-z)^\\alpha}, |z| < 1 < /math > \n * < ref name= " ctcs " / >   < math > \\sum_{k=0}^\\infty \\frac{1}{k+1}{2k \\choose k} z^k = \\frac{1-\\sqrt{1-4z}}{2z}, |z|\\leq\\frac{1}{4} < /math > , generating function of the Catalan numbers \n * < ref name= " ctcs " / >   < math > \\sum_{k=0}^\\infty {2k \\choose k} z^k = \\frac{1}{\\sqrt{1-4z}}, |z| < \\frac{1}{4} < /math > , generating function of the Central binomial coefficients \n * < ref name= " ctcs " / >   < math > \\sum_{k=0}^\\infty {2k + \\alpha \\choose k} z^k = \\frac{1}{\\sqrt{1-4z}}\\left(\\frac{1-\\sqrt{1-4z}}{2z}\\right)^\\alpha, |z| < \\frac{1}{4} < /math > \n \n ===Harmonic numbers=== \n * < math >  \\sum_{k=1}^\\infty H_k z^k = \\frac{-\\ln(1-z)}{1-z}, |z| < 1 < /math > \n * < math >  \\sum_{k=1}^\\infty \\frac{H_k}{k+1} z^{k+1} = \\frac{1}{2}\\left[\\ln(1-z)\\right]^2, \\qquad |z| < 1 < /math > \n * < math >  \\sum_{k=1}^\\infty \\frac{(-1)^{k-1} H_{2k}}{2k+1} z^{2k+1} = \\frac{1}{2} \\arctan{z} \\log{(1+z^2)}, \\qquad |z| < 1  < /math > < ref name= " gfo " / > \n * < math >  \\sum_{n=0}^\\infty \\sum_{k=0}^{2n} \\frac{(-1)^k}{2k+1} \\frac{z^{4n+2}}{4n+2} = \\frac{1}{4} \\arctan{z} \\log{\\frac{1+z}{1-z}},\\qquad  |z| < 1  < /math > < ref name= " gfo " / > \n \n == Binomial coefficients == \n * < math > \\sum_{k=0}^n {n \\choose k} = 2^n < /math > \n * < math > \\sum_{k=0}^n (-1)^k {n \\choose k} = 0, \\text{ where }n > 0 < /math > \n * < math > \\sum_{k=0}^n {k \\choose m} = { n+1 \\choose m+1 } < /math > \n * < math > \\sum_{k=0}^n {m+k-1 \\choose k} = { n+m \\choose n } < /math >  (see Multiset) \n * < math > \\sum_{k=0}^n {\\alpha \\choose k}{\\beta \\choose n-k} = {\\alpha+\\beta \\choose n} < /math >  (see Vandermonde identity) \n \n == Trigonometric functions == \n Sums of sines and cosines arise in Fourier series. \n \n * < math > \\sum_{k=1}^\\infty \\frac{\\sin(k\\theta)}{k}=\\frac{\\pi-\\theta}{2}, 0 < \\theta < 2\\pi < /math > \n * < math > \\sum_{k=1}^\\infty \\frac{\\cos(k\\theta)}{k}=-\\frac{1}{2}\\ln(2-2\\cos\\theta), \\theta\\in\\mathbb{R} < /math > \n * < math > \\sum_{k=0}^\\infty \\frac{\\sin[(2k+1)\\theta]}{2k+1}=\\frac{\\pi-\\theta}{2}, 0 < \\theta < \\pi < /math > < ref > http://www.wolframalpha.com/widgets/view.jsp?id=a787670f0f1047d7fbe288763c55ba14 < /ref > \n * < math > B_n(x)=-\\frac{n!}{2^{n-1}\\pi^n}\\sum_{k=1}^\\infty \\frac{1}{k^n}\\cos\\left(2\\pi kx-\\frac{\\pi n}{2}\\right), 0 < x < 1 < /math > < ref >  < /ref > \n * < math > \\sum_{k=0}^n \\sin(\\theta+k\\alpha)=\\frac{\\sin\\frac{(n+1)\\alpha}{2}\\sin(\\theta+\\frac{n\\alpha}{2})}{\\sin\\frac{\\alpha}{2}} < /math > \n * < math > \\sum_{k=0}^n \\cos(\\theta+k\\alpha)=\\frac{\\sin\\frac{(n+1)\\alpha}{2}\\cos(\\theta+\\frac{n\\alpha}{2})}{\\sin\\frac{\\alpha}{2}} < /math > \n * < math > \\sum_{k=1}^{n-1} \\sin\\frac{\\pi k}{n}=\\cot\\frac{\\pi}{2n} < /math > \n * < math > \\sum_{k=1}^{n-1} \\sin\\frac{2\\pi k}{n}=0 < /math > \n * < math > \\sum_{k=0}^{n-1} \\csc^2\\left(\\theta+\\frac{\\pi k}{n}\\right)=n^2\\csc^2(n\\theta) < /math > < ref >  < /ref > \n * < math > \\sum_{k=1}^{n-1} \\csc^2\\frac{\\pi k}{n}=\\frac{n^2-1}{3} < /math > \n * < math > \\sum_{k=1}^{n-1} \\csc^4\\frac{\\pi k}{n}=\\frac{n^4+10n^2-11}{45} < /math > \n \n == Rational functions == \n * < math > \\sum_{n=a+1}^{\\infty} \\frac{a}{n^2 - a^2} = \\frac{1}{2} H_{2a} < /math > < ref > \n  < /ref > \n * < math > \\sum_{n=0}^\\infty\\frac{1}{n^2+a^2}=\\frac{1+a\\pi\\coth (a\\pi)}{2a^2} < /math > \n * < math > \\displaystyle \\sum_{n=0}^\\infty \\frac {1}{n^4+4a^4} = \\dfrac{1 + a \\pi \\coth (a \\pi)}{8a^4} < /math > \n *An infinite series of any rational function of  < math > n < /math >  can be reduced to a finite series of polygamma functions, by use of partial fraction decomposition. < ref >  < /ref >  This fact can also be applied to finite series of rational functions, allowing the result to be computed in constant time even when the series contains a large number of terms. \n \n ==See also== \n  \n * Series (mathematics) \n * List of integrals \n * Summation \n * Taylor series \n * Binomial theorem \n * Gregory\'s series \n * On-Line Encyclopedia of Integer Sequences \n  \n \n ==Notes== \n  \n \n ==References== \n *Many books with a list of integrals also have a list of series. \n \n Category:Mathematical series \n Series \n Series')

This list of mathematical series contains formulae for finite and infinite sums. It can be used in conjunction with other tools for evaluating sums. 
 *Here,  < math > 0^0 < /math >  is taken to have the value  < math > 1 < /math > 
 * < math > B_n(x) < /math >  is a Bernoulli polynomial. 
 * < math > B_n < /math >  is a Bernoulli number, and here,  < math > B_1=-\frac{1}{2}. < /math > 
 * < math > E_n < /math >  is an Euler number. 
 * < math > \zeta(s) < /math >  is the Riemann zeta function. 
 * < math > \Gamma(z) < /math >  is the gamma function. 
 * < math > \psi_n(z) < /math >  is a polygamma function. 
 * < math > \operatorname{Li}_s(z) < /math >  is a polylogarithm. 
 * < math >  n \choose k  < /math >  is binomial coefficient 
 
 ==Sums of powers== 
 See Faulhaber's formula. 
 * < math > \sum_{k=0}^m k^{n-1}=\frac{B_n(m+1)-B_n}{n} < /math > 
 The first few values are: 
 * < math > \sum_{k=1}^m k=\frac{m(m+1)}{2} < /math > 
 * < math > \sum_{k=1}^m k^2=\frac{m(m+1)(2m+1)}{6}=\f